# Important Note!

This file is for auto annotation of the user uploaded pill images.

Files Created:
- Annotated image of the uploaded image
- txt file containing the location of the bounding box

Files Modified:
- data.yaml file is updated to include the new classes

Algorithm Difference compared to the yolo_predictions.py:
- Hard codes the class (for annotation purpose)


Future Improvements:
- The loop for annotating the images calls the yaml file multiple times. This can result in lag.
- Make the user input detect for pills. If no pill detected, ask them to re-upload a new image.

In [1]:
from PIL import Image
import os
import yaml
import cv2
import Augmentor

In [2]:
help(Augmentor)

Help on package Augmentor:

NAME
    Augmentor - The Augmentor image augmentation library.

DESCRIPTION
    Augmentor is a software package for augmenting image data. It provides a number of utilities that aid augmentation in a automated manner. The aim of the package is to make augmentation for machine learning tasks less prone to error, more reproducible, more efficient, and easier to perform.
    
    .. moduleauthor:: Marcus D. Bloice <marcus.bloice@medunigraz.at>
       :platform: Windows, Linux, Macintosh
       :synopsis: An image augmentation library for Machine Learning.

PACKAGE CONTENTS
    ImageSource
    ImageUtilities
    Operations
    Pipeline

CLASSES
    builtins.object
        Augmentor.Pipeline.Pipeline
            Augmentor.Pipeline.DataFramePipeline
            Augmentor.Pipeline.DataPipeline
    
    class DataFramePipeline(Pipeline)
     |  DataFramePipeline(source_dataframe, image_col, category_col, output_directory='output', save_format=None)
     |  
     |  

In [4]:
import os
import Augmentor

# Function to apply augmentation to each subfolder
def augment_images_in_folder(folder_path, num_augmented_images):
    pipeline = Augmentor.Pipeline(folder_path)
    
    # Define augmentation operations
    pipeline.flip_left_right(probability=0.7)
    pipeline.random_brightness(probability=0.5, min_factor=0.5, max_factor=1.4)
    pipeline.rotate90(probability=0.6)
    pipeline.zoom_random(probability=0.5, percentage_area=0.8)

    pipeline.sample(num_augmented_images)

# Main script
input_folder = 'Testing Image/Upload Image/'

# Iterate through subfolders and apply augmentation
for subfolder in os.listdir(input_folder):
    subfolder_path = os.path.join(input_folder, subfolder)
    
    # Check if it's a directory
    if os.path.isdir(subfolder_path):
        print(f"Augmenting images in folder: {subfolder}")
        augment_images_in_folder(subfolder_path, 100)

print("Augmentation complete.")

Augmenting images in folder: gastric
Initialised with 2 image(s) found.
Output directory set to Testing Image/Upload Image/gastric\output.

Processing <PIL.Image.Image image mode=RGB size=2048x1152 at 0x1D91B0B7700>: 100%|█| 100/100 [00:05<00:00, 17.16 Sample


Augmenting images in folder: heart
Initialised with 2 image(s) found.
Output directory set to Testing Image/Upload Image/heart\output.

Processing <PIL.Image.Image image mode=RGB size=2048x1152 at 0x1D92A9EB1C0>: 100%|█| 100/100 [00:06<00:00, 15.98 Sample


Augmenting images in folder: ibuprofen
Initialised with 2 image(s) found.
Output directory set to Testing Image/Upload Image/ibuprofen\output.

...

Augmenting images in folder: nausea
Initialised with 2 image(s) found.
Output directory set to Testing Image/Upload Image/nausea\output.

Processing <PIL.Image.Image image mode=RGB size=2048x1152 at 0x1D92A9EB580>: 100%|█| 100/100 [00:06<00:00, 16.42 Sample


Augmenting images in folder: panadol
Initialised with 2 image(s) found.
Output directory set to Testing Image/Upload Image/panadol\output.

Processing <PIL.Image.Image image mode=RGB size=2048x1152 at 0x1D92AAB5A60>: 100%|█| 100/100 [00:05<00:00, 17.32 Sample

Augmentation complete.


Augment some images for train test split!

In [6]:
import os
import cv2
import yaml
from test35 import YOLO_Pred  # Make sure to import the YOLO_Pred class from your test35 module

input_root_folder = 'Testing Image/Upload Image/'
output_root_folder = 'Testing Image/Annotated/'
augmented_folder = 'output'  # Adjust this according to the new structure
data_yaml_path = 'data.yaml'

# load the model
yolo = YOLO_Pred('./Model/LD_Model/best.onnx', 'data.yaml')

# update the data.yaml file with new classes
def update_data_yaml(data_yaml_path, new_class):

    if not os.path.exists(data_yaml_path):  # Check if the file exists
        with open(data_yaml_path, 'w') as file:  # Create an empty YAML file with the necessary structure
            yaml.dump({'nc': 0, 'names': []}, file)

    with open(data_yaml_path, 'r') as file:  # Load the existing YAML content
        data = yaml.load(file, Loader=yaml.FullLoader)

    if not data['names']:  # Check if the new class already exists (case-insensitive check)
        data['names'] = [new_class]  # Initialize the data['names'] to be a list
        data['nc'] = 1
        with open(data_yaml_path, 'w') as file:
            yaml.dump(data, file)
        print(f"Class '{new_class}' added to {data_yaml_path}")

    else:  # Append new classes if data['names'] is already a list
        if new_class not in [existing_class.lower() for existing_class in data['names']]:  # Check for duplicate names
            data['names'].append(new_class)  # Add the new class to the list of names
            data['nc'] = len(set([name.lower() for name in data['names']]))  # Update the count of classes

            with open(data_yaml_path, 'w') as file:  # Save the updated YAML content
                yaml.dump(data, file)
            print(f"Class '{new_class}' added to {data_yaml_path}")

        else:
            print(f"Class '{new_class}' already exists in {data_yaml_path}")

    return data['names']

# Main script
for pill_folder in os.listdir(input_root_folder):  # iterate through the image folders
    pill_folder_path = os.path.join(input_root_folder, pill_folder)
    if os.path.isdir(pill_folder_path):  # Check if the folder is a directory

        if pill_folder != augmented_folder:  # Augmented_Photos folder is for storing the augmented uploaded images
            new_class_name = pill_folder  # Use the name of the image folder as the class name
            class_names = update_data_yaml(data_yaml_path, new_class_name)  # Update YAML file
            one_hot_encoding = {class_name: index for index, class_name in enumerate(class_names)}  # Perform one-hot encoding
        
            augmented_folder_path = os.path.join(pill_folder_path, augmented_folder)  # Path to the augmented images
            
            for pic_name in os.listdir(augmented_folder_path):  # Iterate through files in the augmented subfolder
                
                if pic_name.lower().endswith(('.png', '.jpg', '.jpeg')):  # Check if the file is an image
                    img_path = os.path.join(augmented_folder_path, pic_name)  # Adjust the path to the augmented images
                    
                    img = cv2.imread(img_path)  # Load the image
                    
                    original = img.copy()
                    
                    # Check for existing files with the same name
                    existing_count = 0
                    base_name = os.path.splitext(pic_name)[0]  # Remove the extension
                    while True:
                        annotated_txt_path = os.path.join(output_root_folder, pill_folder, f'{base_name}{existing_count}.txt')
                        annotated_img_path = os.path.join(output_root_folder, pill_folder, f'{base_name}{existing_count}.jpeg')

                        if not os.path.exists(os.path.dirname(annotated_txt_path)):
                            os.makedirs(os.path.dirname(annotated_txt_path))

                        if not os.path.exists(os.path.dirname(annotated_img_path)):
                            os.makedirs(os.path.dirname(annotated_img_path))

                        if not os.path.exists(annotated_txt_path) and not os.path.exists(annotated_img_path):
                            break

                        existing_count += 1

                    

                    # img_pred - annotated image, bbox - cx,cy,w,h, classes - detected pill class.
                    img_pred, bbox, classes = yolo.predictions(img, new_class_name)  # Perform pill detection and label

                    values = []
                    if len(bbox) > 0:
                        cv2.imwrite(annotated_img_path, original)
                        x, y, w, h = bbox[0]
                        values.append((x, y, w, h))

                        # Save the normalized center x, center y, width, and height value.
                        # This will be used in the training of the model
                        with open(annotated_txt_path, 'w') as file:
                            for val, class_id in zip(values, classes):
                                line = f"{one_hot_encoding[new_class_name]} {val[0]} {val[1]} {val[2]} {val[3]}\n"
                                file.write(line)

print("Annotation and Augmentation complete.")

Class 'gastric' already exists in data.yaml
Class 'heart' already exists in data.yaml
Class 'ibuprofen' already exists in data.yaml
Class 'nausea' already exists in data.yaml
Class 'panadol' already exists in data.yaml
Annotation and Augmentation complete.


# Merge Folders

In [1]:
import os
import shutil
import pandas as pd

# Set the paths for the original and augmented folders
original_folder = "./"
augmented_folder = "./Augmented_Photos/"
merged_folder = "./Merged_Images/"

# Create the merged folder if it doesn't exist
if not os.path.exists(merged_folder):
    os.makedirs(merged_folder)

# Function to copy images and their corresponding txt files
def copy_images_and_txt(src_folder, dest_folder):
    for root, dirs, files in os.walk(src_folder):
        for filename in files:
            # Check if the file is an image (png, jpg, jpeg, etc.)
            if any(filename.lower().endswith(ext) for ext in ['.png', '.jpg', '.jpeg']):
                image_path = os.path.join(root, filename)
                txt_path = os.path.join(root, filename.rsplit(".", 1)[0] + ".txt")

                # Copy the image to the merged folder
                shutil.copy(image_path, dest_folder)

                # Copy the corresponding txt file if it exists
                if os.path.exists(txt_path):
                    shutil.copy(txt_path, dest_folder)

# Copy original images and txt files
copy_images_and_txt(original_folder, merged_folder)

# Create a Pandas DataFrame to store the data
data = []

# Iterate through the merged folder and read txt files
for root, dirs, files in os.walk(merged_folder):
    for filename in files:
        if filename.lower().endswith('.txt'):
            txt_path = os.path.join(root, filename)
            with open(txt_path, 'r') as txt_file:
                content = txt_file.read().split()
                file_name = os.path.splitext(filename)[0]
                class_label = int(content[0])  # Assuming the class is the first value in the txt file
                data.append([file_name, class_label])

# Create a DataFrame
df = pd.DataFrame(data, columns=['Original_File', 'Class'])

SyntaxError: invalid syntax (2631726403.py, line 31)

In [8]:
df

,Original_File,Class
0,gastric_original_gastric1.jpeg_031fe40f-d269-4...,0
1,gastric_original_gastric1.jpeg_099c3fc3-0d72-4...,0
2,gastric_original_gastric1.jpeg_0cb2e652-5d33-4...,0
3,gastric_original_gastric1.jpeg_0fc3187c-bc49-4...,0
4,gastric_original_gastric1.jpeg_119765ec-b9d8-4...,0
...,...,...
492,panadol_original_panadol2.jpeg_ddcc0429-45ae-4...,4
493,panadol_original_panadol2.jpeg_f340c46c-27fb-4...,4
494,panadol_original_panadol2.jpeg_f4980698-8cc2-4...,4
495,panadol_original_panadol2.jpeg_f535ea14-fa2c-4...,4


In [9]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split

# Set the path for the folder to store train and test data
train_folder = "./Merged_Images/Train_Data/"
test_folder = "./Merged_Images/Test_Data/"

# Create train and test folders if they don't exist
if not os.path.exists(train_folder):
    os.makedirs(train_folder)

if not os.path.exists(test_folder):
    os.makedirs(test_folder)

# Use train_test_split with stratify to maintain class distribution
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['Class'], random_state=42)

# Move images and txt files for the train set
for _, row in train_df.iterrows():
    original_file = row['Original_File']
    image_path = os.path.join(merged_folder, original_file + '.jpeg')
    txt_path = os.path.join(merged_folder, original_file + '.txt')
    shutil.move(image_path, train_folder)
    shutil.move(txt_path, train_folder)

# Move images and txt files for the test set
for _, row in test_df.iterrows():
    original_file = row['Original_File']
    image_path = os.path.join(merged_folder, original_file + '.jpeg')
    txt_path = os.path.join(merged_folder, original_file + '.txt')
    shutil.move(image_path, test_folder)
    shutil.move(txt_path, test_folder)


In [ ]:
# take the trained model and test out on the pills.
# ensure the model can count the occurrence of each pill type.
# after it is done then proj done!!!!!!

In [10]:
import os
os.chdir('./yolo')

In [11]:
ls

 Volume in drive C is OS
 Volume Serial Number is E6EB-988B

 Directory of C:\Users\fangg\OneDrive\Documents\FYP\FSP_IVideoAnalytics\Andrew\Pill-Detection\Notebooks\yolo

11/12/2023  09:00 am    <DIR>          .
12/12/2023  09:24 am    <DIR>          ..
11/12/2023  11:19 am    <DIR>          yolov5
               0 File(s)              0 bytes
               3 Dir(s)  244,090,785,792 bytes free


In [12]:
# !git clone https://github.com/ultralytics/yolov5.git

In [13]:
os.chdir('yolov5')
# pip install Pillow==10.1.0
# pip install -r requirements.txt

In [14]:
# manually download the arial.ttf file and moved to "C:\Users\fangg\AppData\Roaming\Ultralytics"
!python train.py --data user_data.yaml --weights yolov5s.pt --img 640 --batch-size 16 --name Model --epochs 50

train: weights=yolov5s.pt, cfg=, data=user_data.yaml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs\train, name=Model, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github:  YOLOv5 is out of date by 2 commits. Use 'git pull' or 'git clone https://github.com/ultralytics/yolov5' to update.
YOLOv5  v7.0-247-g3f02fde Python-3.9.13 torch-2.0.1+cpu CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0

In [15]:
!python export.py --weights ./runs/train/Model/weights/best.pt --include onnx --simplify --opset 12

============== Diagnostic Run torch.onnx.export version 2.0.1+cpu ==============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



export: data=C:\Users\fangg\OneDrive\Documents\FYP\FSP_IVideoAnalytics\Andrew\Pill-Detection\Notebooks\yolo\yolov5\data\coco128.yaml, weights=['./runs/train/Model/weights/best.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, dynamic=False, simplify=True, opset=12, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLOv5  v7.0-247-g3f02fde Python-3.9.13 torch-2.0.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7023610 parameters, 0 gradients, 15.8 GFLOPs

PyTorch: starting from runs\train\Model\weights\best.pt with output shape (1, 25200, 10) (13.7 MB)

ONNX: starting export with onnx 1.15.0...
ONNX: simplifying with onnx-simplifier 0.4.35...
ONNX: export success  2.3s, saved as runs\train\Model\weights\best.onnx (27.2 MB)

Export complete (3.3s)
Results saved to C:\Users\fangg\OneDrive\Documents\FYP\FSP_IVideoAnalytics

In [ ]:
# # Updating the YAML File
# def update_data_yaml(data_yaml_path, new_class):

#     # Check if the file exists
#     if not os.path.exists(data_yaml_path):
#         # Create an empty YAML file with the necessary structure
#         with open(data_yaml_path, 'w') as file:
#             yaml.dump({'nc': 0, 'names': []}, file)

#     # Load the existing YAML content
#     with open(data_yaml_path, 'r') as file:
#         data = yaml.load(file, Loader=yaml.FullLoader)

#     # Check if the new class already exists (case-insensitive check)
#     if not data['names']:
#         data['names'] = [new_class]
#         data['nc'] = 1
#         with open(data_yaml_path, 'w') as file:
#             yaml.dump(data, file)
#         print(f"Class '{new_class}' added to {data_yaml_path}")
#     else:
#         if new_class not in [existing_class.lower() for existing_class in data['names']]:
#             # Add the new class to the list of names
#             data['names'].append(new_class)

#             # Update the count of classes
#             data['nc'] = len(set([name.lower() for name in data['names']]))

#             # Save the updated YAML content
#             with open(data_yaml_path, 'w') as file:
#                 yaml.dump(data, file)
#             print(f"Class '{new_class}' added to {data_yaml_path}")
#         else:
#             print(f"Class '{new_class}' already exists in {data_yaml_path}")

#     return data['names']

# pill_name = input("Enter name of the pill: ").lower()
# data_yaml_path = 'data.yaml'
# new_class_name = pill_name

# class_names = update_data_yaml(data_yaml_path, new_class_name)
# print("Class names:", class_names)

# # Create a one-hot encoding dictionary
# one_hot_encoding = {class_name: index for index, class_name in enumerate(class_names)}
# print("One-Hot Encoding Dictionary:", one_hot_encoding)




# # Pill Detection
# from testing import YOLO_Pred

# pic_name = input("Enter name of the file: ")

# # load the model
# yolo = YOLO_Pred('./Model/weights/best.onnx', 'data.yaml')

# # img = cv2.imread(f'Testing Image/Upload Image/{pic_name}.jpeg')
# img = cv2.imread(f'Testing Image/Upload Image/{pic_name}')
# encoded_class = one_hot_encoding[pill_name]

# # Check for existing files with the same name
# existing_count = 0
# while True:
#     base_name = f'{pill_name}{existing_count}'
#     annotated_txt_path = f'Testing Image/Annotated/{base_name}.txt'
#     annotated_img_path = f'Testing Image/Annotated/{base_name}.jpeg'

#     if not os.path.exists(annotated_txt_path) and not os.path.exists(annotated_img_path):
#         break

#     existing_count += 1

# # Save annotated image
# cv2.imwrite(annotated_img_path, img)

# # Perform predictions
# img_pred, bbox, index, classes = yolo.predictions(img, pill_name)
# height, width, channels = img_pred.shape
# bboxes = [bbox[ind] for ind in index]

# values = []

# for box in bboxes:
#     x, y, w, h = box
#     center_x = ((x + w) / 2) / width
#     center_y = ((y + h) / 2) / height

#     # width
#     width_normalized = w / width
#     # height
#     height_normalized = h / height

#     values.append((center_x, center_y, width_normalized, height_normalized))

# # Save annotated text file
# with open(annotated_txt_path, 'w') as file:
#     for val, class_id in zip(values, classes):
#         line = f"{encoded_class} {val[0]} {val[1]} {val[2]} {val[3]}\n"
#         file.write(line)

In [2]:
# # Resize the user input 

# def resize_images(input_folder, output_folder, new_size):
#     # Create the output folder if it doesn't exist
#     if not os.path.exists(output_folder):
#         os.makedirs(output_folder)

#     # List all files in the input folder
#     files = os.listdir(input_folder)

#     for file in files:
#         # Check if the file is an image
#         if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
#             # Construct the input and output file paths
#             input_path = os.path.join(input_folder, file)

#             output_path = os.path.join(output_folder, file)

#             # Open the image
#             with Image.open(input_path) as img:
#                 # Calculate new dimensions while maintaining the aspect ratio
#                 width_percent = (new_size[0] / float(img.size[0]))
#                 height_size = int((float(img.size[1]) * float(width_percent)))
#                 new_dimensions = (new_size[0], height_size)

#                 # Resize the image
#                 resized_img = img.resize(new_dimensions)

#                 # Save the resized image to the output folder
#                 resized_img.save(output_path)

# input_folder = "Testing Image/Upload Image/"
# output_folder = "Testing Image/Resized Image/"

# # Set the new size for the images
# new_size = (240, 320)

# # Resize images
# resize_images(input_folder, output_folder, new_size)

# print("Image resizing complete.")

Image resizing complete.
